### Import libraries

In [25]:
import numpy as np
import pandas as pd

### Customize

In [26]:
# Provide name of the CSV file to save/load the raw data
my_raw_data='0505 simulation_raw_data.csv'

# Provide name of the CSV file to save the clean data
my_clean_data= '0505 simulation_clean_data.csv'

# Provide name of the CSV file to save the simulated data
my_simulated_data='0505 simulation_values.csv'

# Provide number of days over which return is to be computed
return_days=10

# Provide number of simulation runs
simulation_runs=10000

# Provide and lower and upper confidence levels 
my_lower_limit=2.5
my_upper_limit=97.5


### Load raw data and compute return

In [27]:
#Load the downloaded file into a DataFrame
raw_data = pd.read_csv(my_raw_data, index_col='Date', parse_dates=True)

# Calculate returns for specified date
raw_data['Return'] = np.log(raw_data['Close'] / raw_data['Close'].shift(return_days))

C:\Users\ahmed\AppData\Local\Temp\ipykernel_2076\3228763819.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  raw_data = pd.read_csv(my_raw_data, index_col='Date', parse_dates=True)


### Clean up and save data

In [28]:
# Drop NaN values in any column
processed_data = raw_data.dropna()

# Save the cleaned DataFrame with Date, Adj Close, and Return
processed_data[['Close', 'Return']].to_csv(my_clean_data, index_label='Date')

### Load data and simulate 

In [29]:
# Load only the 'Return' column from the saved file
data = pd.read_csv(my_clean_data, usecols=['Return'])

# Simulate specified days of returns using historical mean and std dev
simulated_returns = np.random.normal(data['Return'].mean(), data['Return'].std(), simulation_runs)

### Compute simulated values

In [30]:
# Load last close price
last_price = np.round(processed_data['Close'].iloc[-1], 2)

# Apply simulated returns independently to forecast  values
forecasted_values = np.round(last_price * np.exp(simulated_returns), 2)

### Save simulated values

In [31]:
# Create DataFrame for simulations
simulation_df = pd.DataFrame({
    'Simulated Returns': np.round(simulated_returns, 4),  
    'Simulated Index Values': forecasted_values
})

# Confidence intervals
confidence_interval = np.percentile(forecasted_values, [my_lower_limit, my_upper_limit])

# Export to CSV
simulation_df.to_csv(my_simulated_data, index=False)

### Print results

In [32]:
# Print confidence intervals
print(f"Forecasted values based on the last price of {last_price}")
print(f"There is a {my_lower_limit}% probability that price would fall below {confidence_interval[0]:.2f}")
print(f"and a {my_upper_limit}% probability that price would not go above: {confidence_interval[1]:.2f}")


Forecasted values based on the last price of 5886.55
There is a 2.5% probability that price would fall below 5420.58
and a 97.5% probability that price would not go above: 6340.39
